<a href="https://colab.research.google.com/github/michael-rowland/Pitch-Predictions/blob/master/Pitch-Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## NOTES

---

QUESTIONS: 
- should I keep notes in docstring?
- stance mapping 0/1

Features:
- Shifting?
- Home/Away

Per pitcher models

Exponential Smoothing

Permutation Importances

AVERAGE ROC AUC / SCORECARD

MULTIPLE PITCHERS

MODEL TYPES



## SETUP

---



In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    !pip install category_encoders==2.*
    !pip install pybaseball

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [2]:
# import os
# import numpy as np
import pandas as pd
import category_encoders as ce
from pybaseball import statcast_pitcher, playerid_lookup

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## DATA GATHERING

---


For now I am just going to focus on pitchers from my favorite team, the Minnesota Twins. 

Pitchers I will use will come from [here](https://theathletic.com/1570769/2020/01/30/twins-roster-projection-4-0-welcome-bringer-of-rain-and-human-bazooka/), plus recent trade acquisition, Kenta Maeda.

Additional development: grow out dataset to players with X number of innings pitched

In [8]:
berrios = playerid_lookup('berrios', 'jose').loc[0, 'key_mlbam']

Gathering player lookup table. This may take a moment.


In [10]:
raw_data = statcast_pitcher('2008-04-01', '2020-01-01', berrios)

Gathering Player Data


## DATA CLEANING

---



In [22]:
def reorder(df):
  '''
  reorders pitches in ascending order
  '''
  df = df.copy()

  df = df.reindex(index=df.index[::-1])
  df = df.reset_index(drop=True)

  return df

In [23]:
def pitch_cleaning(df):
  '''
  cleans target ('pitch_type'), dropping NaN's (bad data) and non-standard 
  pitches
  '''
  df = df.copy()

  df = df.dropna(subset=['pitch_type'])
  
  df['pitch_type'] = df['pitch_type'].str.lower()
  remove_pitches = ['po', 'in', 'un']
  df = df[~df['pitch_type'].isin(remove_pitches)]
#   df = df.drop('pitch_name', axis=1)

  return df

In [86]:
def historical_cleaning(df):
  '''
  shifts relevant historical data to prevent data leakage
  '''
  df = df.copy()

  df['previous_pitch'] = df['pitch_type'].shift(1)
  df['previous_strike'] = df['type'].shift(1) == 'S'
  df['previous_zone'] = df['zone'].shift(1) # this may need to be encoded somehow
  df = df.drop(['type', 'zone'], axis=1)
  fill_na = ['previous_pitch', 'previous_zone']
  df[fill_na] = df[fill_na].fillna(0)

  return df

## DATA PROCESSING

---


In [25]:
def total_pitches(df):
  '''
  creates column of total pitches prior to current/target pitch
  '''
  df = df.copy()

  df['pitch_total'] = df.groupby('game_date').cumcount(ascending=True)

  return df

In [26]:
def score_margin(df):
  '''
  creates column of score margin for the pitcher's team
  '''
  df = df.copy()

  df['score_margin'] = df['fld_score'] - df['bat_score']
  df = df.drop(['fld_score', 'bat_score'], axis=1)

  return df

In [27]:
def pitcher_advantage(df):
  '''
  creates column of pitcher advantage based on the ball/strike count
  '''
  df = df.copy()

  advantages = {'0-0':'neutral', '0-1':'ahead', '0-2':'ahead', '1-0':'neutral', 
                '1-1':'neutral', '1-2':'ahead', '2-0':'behind', '2-1':'neutral',
                '2-2':'ahead', '3-0':'behind', '3-1':'behind', '3-2':'neutral'}

  df['count'] = df['balls'].astype(str) + '-' + df['strikes'].astype(str)
  df['pitcher_advantage'] = df['count'].map(advantages)
  df = df.drop('count', axis=1)

  return df

In [28]:
def baserunners_mapping(df):
  '''
  maps baserunner to boolean values and creates column if there is a baserunner 
  on any base
  '''
  df = df.copy()

  df['on_1b'] = df['on_1b'].notnull()
  df['on_2b'] = df['on_2b'].notnull()
  df['on_3b'] = df['on_3b'].notnull()
  
  df['baserunner'] = df['on_1b'] | df['on_2b'] | df['on_3b']

  return df

In [29]:
def at_bat_counter(df):
  '''
  creates column of number of batters faced
  '''
  df = df.copy()

  df['at_bat_count'] = df.groupby('at_bat_number').ngroup(ascending=True)+1
  df = df.drop('at_bat_number', axis=1)

  return df

In [93]:
def pitch_encoding(df):
  '''
  creates columns of previous pitch for each pitch type using OneHotEncoder
  '''
  df = df.copy()

  encoder = ce.OneHotEncoder(use_cat_names=True)
  encoded = encoder.fit_transform(df['previous_pitch'])
  df = pd.merge(df, encoded, how='outer', on=df.index)
  df = df.drop(['key_0', 'previous_pitch_0'], axis=1)
  
  return df

In [31]:
def pitch_counter(df):
  '''
  creates columuns of cumulative count of each specific pitch type
  '''
  df = df.copy()

  pitches = [col for col in df if col.startswith('previous_pitch_')]
  
  for pitch in pitches:
    df[f'count_{pitch[-2:]}'] = (df[pitch] == 1).cumsum()

  return df

In [32]:
def pitch_tendency(df, window):
  '''
  creates columns of pitch tendency (mean) over last n = window pitches for 
  each pitch type
  '''
  df = df.copy()

  pitches = [col for col in df if col.startswith('previous_pitch_')]
  
  for pitch in pitches:
#     df[f'{pitch[-2:]}_tendency_{window}'] = df[pitch].iloc[1:].rolling(
    df[f'{pitch[-2:]}_tendency_{window}'] = df[pitch].rolling(
        window, min_periods=1).mean()
    
  return df

In [33]:
def strike_encoding(df):
  '''
  creates columns of strike pitch for each pitch type using OneHotEncoder
  '''
  df = df.copy()
  
  s = df['previous_strike'] == True
  df['strike_pitch'] = s.mask(s, df['previous_pitch'])
  
  encoder = ce.OneHotEncoder(use_cat_names=True)
  encoded = encoder.fit_transform(df['strike_pitch'])
  df = pd.merge(df, encoded, how='outer', on=df.index)
  df = df.drop(['key_0', 'strike_pitch_False', 'strike_pitch'], 
               axis=1, errors='ignore')
  
  return df

In [66]:
def strike_percentage(df, percentage=True):
  '''
  creates columns of strike percentage (or raw count) for each pitch type
  * This could be further engineered to calculate percentage of total number of 
  strikes thrown rather than number of total pitches
  '''
  df = df.copy()

  pitches = [col for col in df if col.startswith('strike_pitch_')]

  for i in pitches:
    target = f'{i[-2:]}_cumulative_strikes'
    if percentage:
      df[target] = (df[f'strike_pitch_{i[-2:]}'] == 1).cumsum() / df['pitch_total']
    else:
      df[target] = (df[f'strike_pitch_{i[-2:]}'] == 1).cumsum()
    df[target] = df[target].fillna(0)
      
  return df

In [35]:
def strike_counter(df, window):
  '''
  creates columns of number of previous n (window) strikes thrown for each pitch 
  type
  '''
  df = df.copy()
  
  pitches = [col for col in df if col.startswith('strike_pitch_')]
  
  for i in pitches:
    target = f'{i[-2:]}_strike_tendency_{window}'
    df[target] = df[df[i] != 0][i].rolling(window, min_periods=1).sum()
    df[target] = df[target].fillna(method='ffill')
    df[target] = df[target].fillna(0)

  return df

In [36]:
def wrangle_game(df):
  '''
  this function does data cleaning and feature engineering on a per game basis
  '''
  df = df.copy()

  # CLEANING FUNCTIONS
  df = reorder(df)
  df = pitch_cleaning(df)
  if df.empty:
    return
  df = historical_cleaning(df)

  # FEATURE ENGINEERING FUNCTIONS
  df = total_pitches(df)
  df = score_margin(df)
  df = pitcher_advantage(df)
  df = baserunners_mapping(df)
  df = at_bat_counter(df)
  df = pitch_encoding(df)
  df = pitch_counter(df)
  
  windows = [5, 10, 20]
  for window in windows:
    df = pitch_tendency(df, window) 
  df = strike_encoding(df)
  df = strike_percentage(df, percentage=True)
  for window in windows:
    df = strike_counter(df, window)
  
  return df

In [37]:
def pitcher_wrangle(pitcher):
  '''
  gathers all available data on specific pitcher (prior to 2020 season)
  '''
  
#   THIS HAS TO BE IN HERE, AND I'M NOT SURE WHY
  target = 'pitch_type'
  features = ['pitch_name', 'pitch_type', 'game_date', 'stand', 'balls', 
              'strikes', 'on_3b', 'on_2b', 'on_1b', 'outs_when_up', 'inning', 
              'at_bat_number', 'bat_score', 'fld_score', 'type', 'zone', 
              'player_name']
  
  raw_data = statcast_pitcher('2008-04-01', '2020-01-01', pitcher)

  df = pd.DataFrame()

  games = raw_data['game_date'].unique().tolist()
  
  for game in games:
    game_data = wrangle_game(raw_data[raw_data['game_date'] == game][features])
    df = df.append(game_data, ignore_index=True, sort=False)
    
  return df

In [94]:
df = pitcher_wrangle(berrios)

Gathering Player Data


## MODEL

---



In [40]:
# MOVE TO TOP WHEN FINISHED
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier

In [41]:
def log_model(txt_file='model_log.txt'):
    with open(txt_file, 'a') as file:
        file.write(f'Validation accuracy: {model.score(X_val_processed, y_val):.5f}\n')
        file.write(f'Features: {X_train.columns}\n')
        file.write(f'{model.get_params}\n')
        file.write('\n')
        file.close()

In [42]:
train, test = train_test_split(df, train_size=0.8, random_state=42)
train, val = train_test_split(train, train_size=0.8, random_state=42)

In [43]:
target = 'pitch_type'
features = df.columns.drop([target, 'game_date'])

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]

In [44]:
processor = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='mean')
)
X_train_processed = processor.fit_transform(X_train)
X_val_processed = processor.transform(X_val)

model = RandomForestClassifier(
                               n_estimators=100, 
                               max_features=16,
                               max_depth=9, 
                               random_state=42, 
                               n_jobs=-1)
model.fit(X_train_processed, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=9, max_features=16,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [45]:
baseline = df['pitch_type'].value_counts(normalize=True)[0]
print(f'Baseline Accuracy: {baseline:.5f}')
print(f'Train Accuracy: {model.score(X_train_processed, y_train):.5f}')
print(f'Validation Accuracy: {model.score(X_val_processed, y_val):.5f}')

Baseline Accuracy: 0.32516
Train Accuracy: 0.65257
Validation Accuracy: 0.42693




---




In [54]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_selection import f_regression, SelectKBest
from scipy.stats import randint, uniform

target = 'pitch_type'
features = df.columns.drop([target, 'game_date'])
train, test = train_test_split(df, train_size=0.8, random_state=0)
X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(), 
#     SelectKBest(f_regression), 
    RandomForestClassifier()
)

param_distributions = {
  'simpleimputer__strategy': ['mean', 'median'], 
#   'selectkbest__k': range(1, len(X_train.columns)+1), 
#   'randomforestclassifier__n_estimators': randint(100, 200), 
  'randomforestclassifier__max_features': range(1, 26), 
  'randomforestclassifier__max_features': uniform(0, 1), 
  'randomforestclassifier__max_depth': [5, 8, 9, 10, 11, 12, 15, 20, None], 
  'randomforestclassifier__min_samples_split': range(2, 11), 
}

search = RandomizedSearchCV(
  pipeline, 
  param_distributions=param_distributions, 
  n_iter=10, 
  cv=5, 
  scoring='accuracy', 
  verbose=10, 
  return_train_score=True, 
  n_jobs=-1
)

In [55]:
search.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   15.2s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   19.8s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   22.0s
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   26.4s
[Parallel(n_jobs=-1)]: Done  41 out of  50 | elapsed:   29.6s remaining:    6.5s
[Parallel(n_jobs=-1)]: Done  47 out of  50 | elapsed:   36.9s remaining:    2.4s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:   37.1s finished


RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=Pipeline(memory=None,
                                      steps=[('ordinalencoder',
                                              OrdinalEncoder(cols=None,
                                                             drop_invariant=False,
                                                             handle_missing='value',
                                                             handle_unknown='value',
                                                             mapping=None,
                                                             return_df=True,
                                                             verbose=0)),
                                             ('simpleimputer',
                                              SimpleImputer(add_indicator=False,
                                                            copy=True,
                                                            fill_value=None,


In [56]:
search.best_params_

{'randomforestclassifier__max_depth': 8,
 'randomforestclassifier__max_features': 0.7890173689815086,
 'randomforestclassifier__min_samples_split': 10,
 'simpleimputer__strategy': 'median'}

In [57]:
search.best_score_

0.41963428340428094

In [95]:
log_df = df
pitch_mapping = {'ff': 1, 'ft': 1, 'cu': 0, 'ch': 0}
log_df['pitch_type'] = log_df['pitch_type'].map(pitch_mapping)

advantage_mapping = {'behind': -1, 'neutral': 0, 'ahead': 1}
log_df['pitcher_advantage'] = log_df['pitcher_advantage'].map(advantage_mapping)

stance_mapping = {'R': 1, 'L': 0}
log_df['stand'] = log_df['stand'].map(stance_mapping)

bool_cols = ['on_3b', 'on_2b', 'on_1b', 'previous_strike', 'baserunner']
log_df[bool_cols] = log_df[bool_cols].astype(int)

drop_cols = ['game_date', 'player_name', 'previous_zone', 'previous_pitch']
log_df = log_df.drop(drop_cols, axis=1)

In [96]:
baseline = log_df['pitch_type'].value_counts(normalize=True).tolist()[0]
baseline

0.59375

In [103]:
target = 'pitch_type'
features = log_df.columns.drop(target)

train, test = train_test_split(log_df, train_size=0.8, random_state=42)
train, val = train_test_split(train, train_size=0.8, random_state=42)

X_train = train[features]
X_val = val[features]

In [105]:
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train, y_train)
accuracy = log_model.score(X_val, y_val)
print(f'Validation Accuracy: {accuracy*100:.2f}%')

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [108]:
for column in log_df.columns:
  isnull = log_df[column].isnull().any()
  print(f'{isnull} - {column}')

False - pitch_type
False - stand
False - balls
False - strikes
False - on_3b
False - on_2b
False - on_1b
False - outs_when_up
False - inning
False - previous_strike
False - pitch_total
False - score_margin
False - pitcher_advantage
False - baserunner
False - at_bat_count
False - previous_pitch_ff
False - previous_pitch_ft
False - previous_pitch_cu
False - previous_pitch_ch
False - count_ff
False - count_ft
False - count_cu
False - count_ch
False - ff_tendency_5
False - ft_tendency_5
False - cu_tendency_5
False - ch_tendency_5
False - ff_tendency_10
False - ft_tendency_10
False - cu_tendency_10
False - ch_tendency_10
False - ff_tendency_20
False - ft_tendency_20
False - cu_tendency_20
False - ch_tendency_20
False - strike_pitch_ff
False - strike_pitch_ft
True - strike_pitch_cu
True - strike_pitch_ch
False - ff_cumulative_strikes
False - ft_cumulative_strikes
True - cu_cumulative_strikes
True - ch_cumulative_strikes
False - ff_strike_tendency_5
False - ft_strike_tendency_5
True - cu_stri

In [113]:
log_df[log_df['strike_pitch_ch'].isnull()].T

,2356,2357,2358,2359,2360,2361,2362,2363,2364,2365,2366,2367,2368,2369,2370,2371,2372,2373,2374,2375,2376,2377,2378,2379,2380,2381,2382,2383,2384,2385,2386,2387,2388,2389,2390,2391,2392,2393,2394,2395,2396,2397,2398,2399,2400,2401,2402,2403,2404,2405,2406,2407,2408,2409,2410,2411,2412,2413,2414,2415,2416,2417,2418,2419,2420,2421,2422,2423,2424,2425,2426,2427,2428,2429,2430,2431,2432,2433,2434,2435,2436,2437,2438,2439,2440,2441,2442,2443,2444,2445,2446,2447,2448,2449,2450,2451,2452,2453,2454,2455,2456,3326,3327,3328,3329,3330,3331,3332,3333,3334,3335,3336,3337,3338,3339,3340,3341,3342,3343,3344,3345,3346,3347,3348,3349,3350,3351,3352,3353,3354,3355,3356,3357,3358,3359,3360,3361,3362,3363,3364,3365,3366,3367,3368,3369,3370,3371,3372,3373,3374,3375,3376,3377,3378,3379,3380,3381,3382,3383,3384,3385,3386,3387,3388,3389,3390,3391,3392,3393,3394,3395,3396,3397,3398,3399,3400,3401,3402,3403,3404,3405,3406,3407,3408,3409,3410,3411,3412,3413,3414,3415,3416,3417,3418,3419,3420,3421,3422,3423,3424,3425,3426,3427,3428,3429,3971,3972,3973,3974,3975,3976,3977,3978,3979,3980,3981,3982,3983,3984,3985,3986,3987,3988,3989,3990,3991,3992,3993,3994,3995,3996,3997,3998,3999,4000,4001,4002,4003,4004,4005,4006,4007,4008,4009,4010,4011,4012,4013,4014,4015,4016,4017,4018,4019,4020,4021,4022,4023,4024,4025,4026,4027,4028,4029,4030,4031,4032,4033,4034,4035,4036,4037,4038,4039,4040,4041,4042,4043,4044,4045,4046,4047,4048,4049,4050,4051,4052,4053,4054,4055,4056,4057,4058,4059,4060,4061,4062,4063,4064,4065,4066,4067,6018,6019,6020,6021,6022,6023,6024,6025,6026,6027,6028,6029,6030,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040,6041,6042,6043,6044,6045,6046,6047,6048,6049,6050,6051,6052,6053,6054,6055,6056,6057,6058,6059,6060,6061,6062,6063,6064,6065,6066,6067,6068,6069,6070,6071,6072,6073,6074,6075,6076,6077,6078,6079,6080,6081,6082,6083,6084,6085,6086,6087,6088,6089,6090,6091,6092,6093,6094,6467,6468,6469,6470,6471,6472,6473,6474,6475,6476,6477,6478,6479,6480,6481,6482,6483,6484,6485,6486,6487,6488,6489,6490,6491,6492,6493,6494,6495,6496,6497,6498,6499,6500,6501,6502,6503,6504,6505,6506,6507,6508,6509,6510,6511,6512,6513,6514,6515,6516,6517,6518,6519,6520,6521,6522,6523,6524,6525,6526,6527,6528,6529,6530,6531,6532,6533,6534,6535,6536,6537,6538,6539,6540,6541,6542,6543,6544,6545,6546,6547,6548,6549,6550,6551,6552,6553,6554,6555,6556,6557,6558,6559,6560,6935,6936,6937,6938,6939,6940,6941,6942,6943,6944,6945,6946,6947,6948,6949,6950,6951,6952,6953,6954,6955,6956,6957,6958,6959,6960,6961,6962,6963,6964,6965,6966,6967,6968,6969,6970,6971,6972,6973,6974,6975,6976,6977,6978,6979,6980,6981,6982,6983,6984,6985,6986,6987,6988,6989,6990,6991,6992,6993,6994,6995,6996,6997,6998,6999,7000,7001,7002,7003,7004,7005,7006,7007,7008,7009,7010,7011,7012,7013,7014,7015,7016,7017,7018,7019,7020,7021,7022,7023,7024,7025,7026,7027,7028,7029,7030,7031,7032,7033,7034,7035,7036,7037,7038,7039,7040,7041,7042,7043,7044,7045,7046,7047,7048,7049,7050,7051,7052,7053,7054,7055,7056,7057,7058,7059,7060,7061,7062,7063,7064,7065,7066,7067,7068,7069,7070,7071,7072,7073,7074,7075,7076,7077,7078,7079,7080,7081,7082,7083,7084,7085,7086,7087,7088,7089,7090,7091,7092,7093,7094,7095,7096,7097,7098,7099,7100,7101,7102,7103,7104,7105,7106,7107,7108,7109,7110,7111,7112,7113,7114,7115,7116,7117,7118,7119,7120,7121,7122,7123,7124,7125,7126,7127,7128,7129,7130,7131,7132,7133,7134,7135,7136,7137,7138,7139,7140,7141,7142,7143,7144,7145,7146,7147,7148,7149,7150,7151,7152,7153,7154,7155,7156,7157,7158,7159,7160,7161,7162,7163,7164,7165,7166,7167,7168,7169,7170,7171,7172,7173,7174,7175,7176,7177,7178,7179,7180,7181,7182,7183,7184,7185,7186,7187,7188,7189,7190,7191,7192,7193,7194,7195,7196,7197,7198,7199,7200,7201,9232,9233,9234,9235,9236,9237,9238,9239,9240,9241,9242,9243,9244,9245,9246,9247,9248,9249,9250,9251,9252,9253,9254,9255,9256,9257,9258,9259,9260,9261,9262,9263,9264,9265,9266,9267,9268,9269,9270,9271,9272,9273,9274
pitch_type,1,1,0,1,1,0,1,1,0,1,1,1,1,1,0,0,0,1,1,0,1,0,1,0,0,1,0,1,1,1,1,0,1,0,0,1,0

## MISC

---



In [ ]:
def pitch_map(df):
  '''
  generates dictionary mapping 'pitch_type' to 'pitch_name'
  '''
  df = df.copy()

  df = df.dropna(subset=['pitch_type', 'pitch_name'])
  pitch_types = df['pitch_name'].unique().tolist()
  pitch_types_abv = [i.lower() for i in df['pitch_type'].unique().tolist()]
  
  return dict(zip(pitch_types_abv, pitch_types))

# sample = 'https://raw.githubusercontent.com/michael-rowland/Pitch-Predictions/master/sample.csv'
# df = pd.read_csv(sample)
# df = df[df['game_date'] == '2018-09-28']

In [ ]:
# import time

# twins_pitchers = [
#   ['jose', 'berrios'], 
#   ['jake', 'odorizzi'], 
#   ['homer', 'bailey'], 
#   ['lewis', 'thorpe'], 
#   ['randy', 'dobnak'], 
#   ['kenta', 'maeda'], 
#   ['taylor', 'rogers'], 
#   ['sergio', 'romo'], 
#   ['tyler', 'duffey'], 
#   ['trevor', 'may'], 
#   ['tyler', 'clippard'], 
#   ['zack', 'littell'], 
#   ['matt', 'wisler']
# ]

# for pitcher in twins_pitchers[5:6]:
#   print(f'{pitcher[0]} {pitcher[1]}')
#   start = time.time()
#   lookup = playerid_lookup(pitcher[1], pitcher[0])
#   pitcher_id = lookup.loc[0, 'key_mlbam']
#   pitcher.append(pitcher_id)
#   df = pitcher_wrangle(pitcher_id)
#   pitcher.append(df)
#   end = time.time()
#   print(f'Time: {end - start}')
#   print(lookup)